In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import sys
import os
import pickle
import random
import gc
import json

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from ASTROMER.models import SingleBandEncoder
from ASTROMER.preprocessing import make_pretraining
from sklearn.metrics import accuracy_score, f1_score

sys.path.append('..')
from env_config import DATA_PATH, PROJECT_PATH
from ml import get_train_data, get_train_matrices, run_experiments
from ztf import ZTF_DATES, ZTF_LAST_DATES
from astromer import build_model
from features import add_colors
from light_curves import limit_date, preprocess_ztf_light_curves, add_lc_stats
from report import print_summary_table

In [3]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 100
sns.set_style('ticks', {'axes.grid': False})
pd.set_option('mode.chained_assignment', None)
%matplotlib inline

/tmp/ipykernel_883667/781861970.py:2: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [4]:
ztf_date = ZTF_DATES['DR 20']
filters = ['g']

# Read data

In [5]:
# Read the train data
data = {}
for filter in filters:
    _, data[filter] = \
        get_train_data(ztf_date=ztf_date, filter=filter, return_light_curves=False)

In [6]:
# Add lightcurve related information to the dataframes
for filter in filters:
    ztf_x_sdss[filter] = add_lc_stats(ztf_x_sdss[filter])

Adding light curve statistics:   0%|          | 0/2588221 [00:00<?, ?it/s]

In [12]:
for filter in filters:
    for col in tqdm(['mag median', 'mag err mean', 'n obs', 'timespan', 'cadence mean', 'cadence median', 'cadence plus sigma', 'cadence minus sigma']):
        sdss_x_ztf[filter][col] = [lc_dict[col] for lc_dict in ztf_x_sdss[filter]]

  0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
# Get PS and WISE colors
for filter in filters:
    data[filter], features_dict = add_colors(data[filter])

# Run experiments

In [7]:
feature_compositions = [
    # Baseline
    # ['AstrmClf'],
    # Cross match comparisons
    ['AstrmClf', 'PS'],
    # ['AstrmClf', 'WISE'],
    # ['AstrmClf', 'GAIA'],
    # Road to the best ensemble classification
    # ['AstrmClf', 'PS', 'WISE'],
    # ['AstrmClf', 'PS', 'GAIA'],
    # ['AstrmClf', 'PS', 'WISE', 'GAIA'],
]

for filter in filters:
    for feature_labels in tqdm(feature_compositions):
        data_labels = [label for label in feature_labels if label != 'AstrmClf']
        if 'ZTF' not in data_labels:
            data_labels = ['ZTF'] + data_labels
        data_label = '_'.join(data_labels)

        # Run the expeirments for given data composition and almost all feature combinations
        preds, classifiers = run_experiments(
            feature_labels, data[filter], features_dict, filter, ztf_date,
        )

        # Save results
        file_name = '{}-band__{}__RF'.format(filter, data_label)
        file_path = os.path.join(PROJECT_PATH, 'outputs/preds/ZTF_{}'.format(ztf_date), file_name + '__test.csv')
        preds.to_csv(file_path, index=False)
        print('Preds saved to: {}'.format(file_path))

        # Create dict like structure for features
        feature_importances = {}

        # Iterate classifiers
        for key, classifier in classifiers.items():
            feature_labels = key.split(' + ')
            feature_importances[key] = {
                'features': np.concatenate([features_dict[feature_label] for feature_label in feature_labels]).tolist(),
                'importances': classifier.feature_importances_.tolist(),
            }

        # Save feature importances
        file_path = os.path.join(PROJECT_PATH, 'outputs/feature_importance/ZTF_{}'.format(ztf_date), file_name + '.json')
        out_file = open(file_path, 'w')
        json.dump(feature_importances, out_file, indent=4)
        out_file.close()
        print('Feature importances saved to: {}'.format(file_path))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Preds saved to: /home/sjnakoneczny/workspace/ztf-agn/outputs/preds/ZTF_20240117/g-band__ZTF_PS__RF__test.csv
Feature importances saved to: /home/sjnakoneczny/workspace/ztf-agn/outputs/feature_importance/ZTF_20240117/g-band__ZTF_PS__RF.json


# Load results

In [8]:
# Define data compositions
data_compositions = [
    # ['ZTF'],
    ['ZTF', 'PS'],
#     ['ZTF', 'WISE'],
#     ['ZTF', 'GAIA'],
#     ['ZTF', 'PS', 'WISE'],
#     ['ZTF', 'PS', 'GAIA'],
#     ['ZTF', 'PS', 'WISE', 'GAIA'],
]

# Resulting data structures
results_dict = {}
feature_importance_dict = {}

for filter in filters:
    results_dict[filter] = {}
    feature_importance_dict[filter] = {}

    for data_composition in data_compositions:
        data_label = '_'.join(data_composition)

        # Read predictions
        file_name = '{}-band__{}__RF'.format(filter, data_label)
        file_path = os.path.join(PROJECT_PATH, 'outputs/preds/ZTF_{}'.format(ztf_date), file_name + '__test.csv')
        results_dict[filter][data_label] = pd.read_csv(file_path)
        print('Loaded results: {}'.format(file_path))

        # Read feature importances
        file_path = os.path.join(PROJECT_PATH, 'outputs/feature_importance/ZTF_{}'.format(ztf_date), file_name + '.json')
        with open(file_path) as file:
            feature_importance_dict[filter][data_label] = json.load(file)
        print('Loaded feature importances: {}'.format(file_path))

        # Check if Astromer present and add as well
        file_name = 'outputs/preds/ZTF_{}/ZTF_{}__band_{}__xmatch_{}__astromer_FC-1024-512-256'.format(ztf_date, ztf_date, filter, data_label)
        file_name += '__test.csv'
        file_path = os.path.join(PROJECT_PATH, file_name)
        if os.path.exists(file_path):
            df_preds = pd.read_csv(file_path)
            prediction_label = 'y_pred Astrm'
            results_dict[filter][data_label][prediction_label] = df_preds['y_pred']
            print('Loaded Astromer: {}'.format(file_path))

Loaded results: /home/sjnakoneczny/workspace/ztf-agn/outputs/preds/ZTF_20240117/g-band__ZTF_PS__RF__test.csv
Loaded feature importances: /home/sjnakoneczny/workspace/ztf-agn/outputs/feature_importance/ZTF_20240117/g-band__ZTF_PS__RF.json


In [9]:
# Just a test

In [11]:
labels = [
    ('ZTF_PS', [
        'AstrmClf',
        'PS',
        'AstrmClf + PS',
    ]),
]

In [18]:
results_dict['g']['ZTF_PS']['y_true'] = results_dict['g']['ZTF_PS']['CLASS']

In [19]:
print_summary_table(results_dict, labels=labels, filters=['g'], concise=True)

,data,features,QSO support (%) g-band,QSO f1 g-band,QSO f1 global g-band,accuracy g-band
0,ZTF_PS,AstrmClf,1.00,0.88,0.88,0.88
1,ZTF_PS,PS,1.00,0.84,0.84,0.89
2,ZTF_PS,AstrmClf + PS,1.00,0.95,0.95,0.96


\begin{tabular}{llrrrr}
\toprule
  data &      features &  QSO support (%) g-band &  QSO f1 g-band &  QSO f1 global g-band &  accuracy g-band \\
\midrule
ZTF_PS &      AstrmClf &                    1.00 &           0.88 &                  0.88 &             0.88 \\
ZTF_PS &            PS &                    1.00 &           0.84 &                  0.84 &             0.89 \\
ZTF_PS & AstrmClf + PS &                    1.00 &           0.95 &                  0.95 &             0.96 \\
\bottomrule
\end{tabular}



/home/sjnakoneczny/workspace/ztf-agn/notebooks/../report.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(escape=False, na_rep='', float_format='%.2f', index=False))


# Results

## Comparison plots

In [ ]:
args = [
    'mag_median', 'redshift',
    'n_obs', 'n_obs_200',
    'timespan', 'timespan_200',
    'cadence_mean', 'cadence_mean_200',
    'cadence_median', 'cadence_median_200',
    'cadence_std', 'cadence_std_200',
]

preds = [
    'y_pred ZTF',
    'y_pred Astrm',
    'y_pred PS',
]

In [ ]:
for x in args:
    plot_results_as_function(results_dict['g']['ZTF_PS'], x, labels=preds)

In [ ]:
for x in args:
    plot_results_as_function(results_dict['r']['ZTF_PS'], x, labels=preds)

In [ ]:
preds = [
    'y_pred ZTF',
    'y_pred Astrm',
    'y_pred PS',
    'y_pred WISE',
    'y_pred GAIA',
]

In [ ]:
for x in args:
    plot_results_as_function(results_dict['g']['ZTF_PS_WISE_GAIA'], x, labels=preds)

In [ ]:
for x in args:
    plot_results_as_function(results_dict['r']['ZTF_PS_WISE_GAIA'], x, labels=preds)

## Tables

In [ ]:
# Add the master ensemble results
for filter in filters:
    features_val = ztf_x_sdss_features[filter].loc[ztf_x_sdss_features[filter]['is_test'] == True].reset_index(drop=True)
    features_val = features_val.dropna(subset=FEATURES_DICT['ZTF']).reset_index(drop=True)
    
    for feature_hierarchy in feature_hierarchies:
        # Start with Astromer predictions
        preds = results_dict[filter]['ZTF']['y_pred Astrm'].to_numpy().copy()
        
        # Add PS where available
        features_list = np.concatenate([FEATURES_DICT[label] for label in ['ZTF', 'PS']])
        indices = features_val.dropna(subset=features_list).index
        preds[indices] = results_dict[filter]['ZTF_PS']['y_pred AstrmClf + PS'].to_numpy().copy()
        results_dict[filter]['ZTF']['y_pred Astrm + PS'] = preds.copy()

        # Get indices for WISE and GAIA
        indices = {}
        for to_process in [['WISE'], ['GAIA'], ['WISE', 'GAIA']]:
            features_list = np.concatenate([FEATURES_DICT[label] for label in ['ZTF', 'PS'] + to_process])
            indices[' + '.join(to_process)] = features_val.dropna(subset=features_list).index

        # Add WISE and GAIA to ZTF + PS
        for to_process in ['WISE', 'GAIA']:
            preds_tmp = preds.copy()
            preds_tmp[indices[to_process]] = results_dict[filter]['ZTF_PS_' + to_process]['y_pred AstrmClf + PS + ' + to_process].to_numpy().copy()
            results_dict[filter]['ZTF']['y_pred Astrm + PS + ' + to_process] = preds_tmp.copy()
            
        # Add WISE + GAIA to ZTF + PS
        for to_process in ['WISE', 'GAIA']:
            preds[indices[to_process]] = results_dict[filter]['ZTF_PS_' + to_process]['y_pred AstrmClf + PS + ' + to_process].to_numpy().copy()
        preds[indices['WISE + GAIA']] = results_dict[filter]['ZTF_PS_WISE_GAIA']['y_pred AstrmClf + PS + WISE + GAIA'].to_numpy().copy()
        results_dict[filter]['ZTF']['y_pred Astrm + PS + WISE + GAIA'] = preds.copy()

In [ ]:
labels = [
    ('ZTF', [
        'Astrm',
        'AstrmClf',
        'ZTF',
        'AstrmClf + ZTF',
        'Astrm + PS',
        'Astrm + PS + WISE',
        'Astrm + PS + GAIA',
        'Astrm + PS + WISE + GAIA',
    ]),
    ('ZTF_PS', [
        'AstrmClf',
        'PS',
        'AstrmClf + PS',
    ]),
    ('ZTF_WISE', [
        'AstrmClf',
        'WISE',
        'AstrmClf + WISE',
    ]),
    ('ZTF_GAIA', [
        'AstrmClf',
        'GAIA',
        'AstrmClf + GAIA'
    ]),
]

In [ ]:
# For publication
print_summary_table(results_dict, labels=labels, filters=['g', 'r'], concise=True)

In [ ]:
# Details for g band
print_summary_table(results_dict, labels=labels, filters=['g'], concise=False)

In [ ]:
# Details for r band
print_summary_table(results_dict, labels=labels, filters=['r'], concise=False)

## ZTF

In [ ]:
data_label = 'ZTF'
filter = 'g'
make_reports(results_dict, feature_importance_dict, filter=filter, data_label=data_label)

In [ ]:
data_label = 'ZTF'
filter = 'r'
make_reports(results_dict, feature_importance_dict, filter=filter, data_label=data_label)

## ZTF x PS

In [ ]:
data_label = 'ZTF_PS'
filter = 'g'
make_reports(results_dict, feature_importance_dict, filter=filter, data_label=data_label)

In [ ]:
data_label = 'ZTF_PS'
filter = 'r'
make_reports(results_dict, feature_importance_dict, filter=filter, data_label=data_label)

## ZTF x PS x WISE x GAIA

In [ ]:
data_label = 'ZTF_PS_WISE_GAIA'
filter = 'g'
make_reports(results_dict, feature_importance_dict, filter=filter, data_label=data_label,
             feature_labels=['ZTF + AstrmClf + PS + WISE + GAIA'])

# DR 5 vs DR 20 models

In [4]:
drs = ['DR 5', 'DR 20']
filter = 'g'

## Make validation batches

In [5]:
# Get the validation data
X, y = {}, {}

In [6]:
# Standard validation data
for dr in drs:
    ztf_x_sdss, sdss_x_ztf = get_train_data(ztf_date=ZTF_DATES[dr], filter='g', return_features=False)
    _, X[dr], _, y[dr] = get_train_matrices(ztf_x_sdss, sdss_x_ztf)

  0%|          | 0/300593 [00:00<?, ?it/s]

  0%|          | 0/490038 [00:00<?, ?it/s]

In [ ]:
# The DR 20 data but limited to the last date of DR 5
dr = 'DR 20'
last_date = ZTF_LAST_DATES['DR 5']
label = 'DR 20 - lim DR 5'

ztf_x_sdss, sdss_x_ztf = get_train_data(ztf_date=ZTF_DATES[dr], filter='g', return_features=False)

# Linmit the dates
ztf_x_sdss = limit_date(ztf_x_sdss, last_date)

# Get at least 21 observations
n_obs = [len(lc_dict['mjd']) for lc_dict in ztf_x_sdss]
idx = np.array(n_obs) > 20
ztf_x_sdss = np.array(ztf_x_sdss)[np.where(idx)]
sdss_x_ztf = sdss_x_ztf.loc[idx].reset_index(drop=True)

_, X[label], _, y[label] = get_train_matrices(ztf_x_sdss, sdss_x_ztf)

In [27]:
# The DR 5 data with features
label = 'DR 5 - features'
for dr in ['DR 5']:
    ztf_x_sdss, sdss_x_ztf, _ = get_train_data(ztf_date=ZTF_DATES[dr], filter='g',
                                               data_subsets=['ZTF'], return_features=True)
    _, X[label], _, y[label] = get_train_matrices(ztf_x_sdss, sdss_x_ztf)

  0%|          | 0/300581 [00:00<?, ?it/s]

In [32]:
# Data sizes
for label in X:
    print(label, len(X[label]))

DR 5 99196
DR 20 161713
DR 20 - lim DR 5 136343
DR 5 - features 99191


In [6]:
# Make batches
batches = {}
for label in X:
    batches[label] = make_pretraining(
        X[label], labels=y[label], n_classes=3, batch_size=64, shuffle=False,
        sampling=True, max_obs=200, msk_frac=0., rnd_frac=0., same_frac=0., repeat=1,
    )

[INFO] Loading Numpy
[INFO] no masking
[INFO] Loading Numpy


## Run tests

In [7]:
# Get the models
models = {}
for dr in drs:
    date = ZTF_DATES[dr]
    
    astromer = SingleBandEncoder()
    astromer = astromer.from_pretraining('ztfg')
    astromer_encoder = astromer.model.get_layer('encoder')
    classifier = build_model(astromer_encoder, n_classes=3, maxlen=astromer.maxlen, train_astromer=False)
    path_astromer = 'outputs/models/ZTF_{}/ZTF_{}__band_{}__xmatch_ZTF__astromer_FC-1024-512-256'.format(date, date, filter)
    classifier.load_weights(os.path.join(PROJECT_PATH, path_astromer))

    models[dr] = classifier

[INFO] Weights already downloaded
[INFO] Weights already downloaded


In [ ]:
# Make preds
preds = {}

In [8]:
# First turn
for data_label in batches:
    preds[data_label] = {}
    for model_dr in drs:
        preds[data_label][model_dr] = models[model_dr].predict(batches[data_label])

2527/2527 [==============================] - 155s 61ms/step


In [30]:
# Second turn
for data_label in ['DR 20 - lim DR 5', 'DR 5 - features']:
    preds[data_label] = {}
    for model_dr in drs:
        preds[data_label][model_dr] = models[model_dr].predict(batches[data_label])

1550/1550 [==============================] - 95s 61ms/step


In [33]:
# Test the models
for data_label in preds:
    for model_dr in drs:
        # Make preds
        y_pred = preds[data_label][model_dr]
        y_class = np.argmax(y_pred, 1)

        # Print results
        acc = np.round(accuracy_score(y[data_label], y_class), 2)
        f1 = np.round(f1_score(y[data_label], y_class, average=None), 2)
        print('Data {}, model {}, acc. {}, F1 {}'.format(data_label, model_dr, acc, f1))

Data DR 5, model DR 5, acc. 0.92, F1 [0.94 0.89 0.9 ]
Data DR 5, model DR 20, acc. 0.85, F1 [0.88 0.82 0.8 ]
Data DR 20, model DR 5, acc. 0.65, F1 [0.63 0.71 0.64]
Data DR 20, model DR 20, acc. 0.85, F1 [0.87 0.83 0.84]
Data DR 20 - lim DR 5, model DR 5, acc. 0.82, F1 [0.84 0.76 0.82]
Data DR 20 - lim DR 5, model DR 20, acc. 0.78, F1 [0.8  0.73 0.78]
Data DR 5 - features, model DR 5, acc. 0.92, F1 [0.94 0.89 0.9 ]
Data DR 5 - features, model DR 20, acc. 0.85, F1 [0.88 0.82 0.8 ]
